In [1]:
import numpy as np
from sklearn import datasets, model_selection, svm, metrics
from sklearn.datasets import fetch_mldata
from sklearn.neural_network import MLPClassifier
# URL:https://qiita.com/nvtomo1029/items/200dbfe5928c4a0cd625

In [2]:
mnist = datasets.fetch_mldata('MNIST original', data_home='data/src/download/')

print(type(mnist))
print(mnist.keys())

<class 'sklearn.utils.Bunch'>
dict_keys(['DESCR', 'COL_NAMES', 'target', 'data'])


C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
mnist_data = mnist.data / 255 # 8bitのデータを0.0～1.0に正規化
mnist_label = mnist.target

## Data split

In [4]:
data_train, data_test, label_train, label_test = model_selection.train_test_split(mnist_data, mnist_label, test_size=0.05, train_size=0.3) # 学習用は全データの30%、テスト用は5%に設定
print('train_size:', data_train.shape[0]) # 学習データ数
print('test_size:', data_test.shape[0]) # テストデータ数

train_size: 21000
test_size: 3500


In [5]:
classifiers = [
    (('Adam_1layer', MLPClassifier(solver="adam", early_stopping=True))), 
    (('Adam_2layer)', MLPClassifier(solver="adam", early_stopping=True, hidden_layer_sizes=(100, 10)))), 
    (('Adam_3layer)', MLPClassifier(solver="adam",  early_stopping=True, hidden_layer_sizes=(100, 100, 10)))), 
    (('Adam_4layer)', MLPClassifier(solver="adam",  early_stopping=True, hidden_layer_sizes=(100, 100, 100, 10)))) 
]

In [6]:
classifier_names = [
    'Adam', 
    'Adam_2layer', 
    'Adam_3layer', 
    'Adam_4layer' 
]

In [ ]:
for name, clf in classifiers:
    print(name, 'start')
    # 予測
    pre = clf.predict(data_test)
    ### 評価 ###
    # accuracy
    ac_score = metrics.accuracy_score(label_test, pre)
    
    # confusion_matrix
    co_mat = metrics.confusion_matrix(label_test, pre)
    
    # precision, recall, f1-score の評価
    cl_repo = metrics.classification_report(label_test, pre)
    
    print('AC Score:', ac_score)
    print('confusion matrix:')
    print(co_mat)
    print('classification report:')
    print(cl_repo)
    
    print(name, 'end')

## parameter

In [8]:
# ハイパーパラメータのリスト
tuned_parameters = [
    {
        # 最適化手法
        "solver":["lbfgs", "sgd", "adam"], 
        # 隠れ層の層の数と、各層のニューロンの数
        "hidden_layer_sizes":[(100,), (100, 10), (100, 100, 10), (100, 100, 100, 10)], 
    }
]

# ニューラルネットワークの分類器とハイパーパラメータのリストを定義
licv=model_selection.GridSearchCV(MLPClassifier(early_stopping=True), param_grid=tuned_parameters, scoring="accuracy")

In [ ]:
# 最適なハイパーパラメータを探索する
licv.fit(data_train, label_train)

# 最適なパラメータを表示(リストで指定している項目のみ)
print('licv.best_params_ : ', licv.best_params_)

# 最適なパラメータを表示(リストで指定していない項目も含め全項目表示)
print('licv.best_estimator_ : ', licv.best_estimator_)

# 最適なパラメータ指定時の評価値
print('licv.best_score_ : ', licv.best_score_)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
# 求められた最適なパラメータで予測、評価
pre = licv.predict(data_test)
# 正解率取得
ac_score = metrics.accuracy_score(label_test, pre)
print('AC Score:', ac_score)
# 混同行列の出力
co_mat = metrics.confusion_matrix(label_test, pre)
print('confusion matrix:')
print(co_mat)
# precision, recall, f1-score の評価
cl_repo = metrics.classification_report(label_test, pre)
print('classification report:')
print(cl_repo)